In [3]:
import requests
import os
from datasets import load_dataset
import pandas as pd
# Load the dataset from Hugging Face
dataset = load_dataset("PatronusAI/financebench")

path_to_csv_dataset = "financebench_sample_150.csv" 
df = pd.read_csv(path_to_csv_dataset)

In [62]:
import datasets
import tempfile
import logging
import random
import config
import os
import yaml
import time
import torch
import transformers
import pandas as pd
import jsonlines

from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import TrainingArguments
from transformers import AutoModelForCausalLM
from llama import BasicModelRunner

In [57]:
!pip install utilities-package

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Obtaining dependency information for configparser from https://files.pythonhosted.org/packages/81/a3/0e5ed11da4b7770c15f6f319abf053f46b5a06c7d4273c48469b7899bd89/configparser-6.0.0-py3-none-any.whl.metadata
  Obtaining dependency information for unidecode from https://files.pythonhosted.org/packages/e4/63/7685ef40c65aba621ccd2524a24181bf11f0535ab1fdba47e40738eacff6/Unidecode-1.3.7-py3-none-any.whl.metadata
  Obtaining dependency information for uritools from https://files.pythonhosted.org/packages/6b/ff/b16f225ceeb47f5d8899371ce446a8d6c1fe509a8882998b869f2a794c25/uritools-4.0.2-py3-none-any.whl.metadata
  Obtaining dependency information for simplejson from https://files.pythonhosted.org/packages/00/c2/d65b3afd81e1277812a5843ec6041f32194b87889a9f6e79f136a51ee5ef/simplejson-3.19.2-cp3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.79 requires packaging==23.1, but you have packaging 23.2 which is incompatible.
autotrain-advanced 0.6.79 requires werkzeug==2.3.6, but you have werkzeug 3.0.1 which is incompatible.


In [24]:
from pprint import pprint
from transformers import AutoTokenizer

In [4]:
df

,financebench_id,doc_name,doc_link,doc_period,question_type,question,answer,evidence_text,page_number
0,financebench_id_03029,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,What is the FY2018 capital expenditure amount ...,$1577.00,Table of Contents \r\n3M Company and Subsidiar...,60
1,financebench_id_04672,3M_2018_10K,https://investors.3m.com/financials/sec-filing...,2018,metrics-generated,Assume that you are a public equities analyst....,$8.70,Table of Contents \r\n3M Company and Subsidiar...,58
2,financebench_id_00499,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,Is 3M a capital-intensive business based on FY...,"No, the company is managing its CAPEX and Fixe...",3M Company and Subsidiaries\r\n Consolidated S...,"48,50,52"
3,financebench_id_01226,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,domain-relevant,What drove operating margin change as of FY202...,Operating Margin for 3M in FY2022 has decrease...,"SG&A, measured as a percent of sales, increase...",27
4,financebench_id_01865,3M_2022_10K,https://investors.3m.com/financials/sec-filing...,2022,novel-generated,"If we exclude the impact of M&A, which segment...",The consumer segment shrunk by 0.9% organically.,Worldwide Sales Change\r\nBy Business Segment ...,25
...,...,...,...,...,...,...,...,...,...
145,financebench_id_00215,VERIZON_2022_10K,https://www.verizon.com/about/sites/default/fi...,2022,domain-relevant,Is Verizon a capital intensive business based ...,Yes. Verizon's capital intensity ratio was app...,Consolidated Balance Sheets \r\nVerizon Commun...,"56, 23"
146,financebench_id_00566,VERIZON_2022_10K,https://www.verizon.com/about/sites/default/fi...,2022,domain-relevant,Has Verizon increased its debt on balance shee...,No. Verizon's debt decreased by $229 million.,"At December 31, Maturities \r\nInterest \r\nRa...",77
147,financebench_id_06247,WALMART_2018_10K,https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...,2018,metrics-generated,What is FY2018 days payable outstanding (DPO) ...,42.69,Walmart Inc.\r\nConsolidated Statements of Inc...,"57,59"
148,financebench_id_04784,WALMART_2019_10K,https://d18rn0p25nwr6d.cloudfront.net/CIK-0000...,2019,metrics-generated,Based on the information provided primarily in...,0.2%,Walmart�Inc.\r\nConsolidated�Statements�of�Inc...,48


In [5]:
examples = instruction_dataset_df.to_dict()

if "question" in examples and "answer" in examples:
  text = examples["question"][0] + examples["answer"][0]
elif "instruction" in examples and "response" in examples:
  text = examples["instruction"][0] + examples["response"][0]
elif "input" in examples and "output" in examples:
  text = examples["input"][0] + examples["output"][0]
else:
  text = examples["text"][0]


prompt_template_with_input = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:"""

In [ ]:
text = examples["question"][0] + examples["answer"][0] 

In [ ]:
processed_prompt = prompt_template_without_input.format(instruction=j["instruction"])

In [11]:
prompt_template = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{instruction}

### Input:
{input}

### Response:
{response}
"""

In [12]:
def fill_template(row):
    return prompt_template.format(instruction=row['question'], input=row['evidence_text'], response=row['answer'])

# Apply the function to each row in the dataframe
filled_prompts = df.apply(fill_template, axis=1)

In [13]:
filled_prompts[0]

'Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n\n### Instruction:\nWhat is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.\n\n### Input:\nTable of Contents \r\n3M Company and Subsidiaries\r\nConsolidated Statement of Cash Flow s\r\nYears ended December 31\r\n \r\n(Millions)\r\n    \r\n2018\r\n    \r\n2017\r\n    \r\n2016\r\n \r\nCash Flows from Operating Activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nNet income including noncontrolling interest\r\n \r\n$\r\n5,363  \r\n$\r\n4,869  \r\n$\r\n5,058  \r\nAdjustments to reconcile net income including noncontrolling interest to net cash\r\nprovided by operating activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nDepreciation and amortization\r\n \r\n \r\n1,488  \r\n \r\n1,544  \r\n \r\n1,474  \r\nCompany pension and post

In [21]:
finetuning_data = df[['question', 'evidence_text', 'answer']].rename(columns={
    'question': 'instruction',
    'evidence_text': 'input',
    'answer': 'output'
}).to_dict(orient='records')

In [22]:
finetuning_data[0]

{'instruction': 'What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.',
 'input': 'Table of Contents \r\n3M Company and Subsidiaries\r\nConsolidated Statement of Cash Flow s\r\nYears ended December 31\r\n \r\n(Millions)\r\n    \r\n2018\r\n    \r\n2017\r\n    \r\n2016\r\n \r\nCash Flows from Operating Activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nNet income including noncontrolling interest\r\n \r\n$\r\n5,363  \r\n$\r\n4,869  \r\n$\r\n5,058  \r\nAdjustments to reconcile net income including noncontrolling interest to net cash\r\nprovided by operating activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nDepreciation and amortization\r\n \r\n \r\n1,488  \r\n \r\n1,544  \r\n \r\n1,474  \r\nCompany pension and postretirement contributions\r\n \r\n \r\n(370) \r\n \r\n(967) \r\n \r\n(383) \r\nCompany pension and postretirement expense\r\n \r\n \r\n410  \r\n \r\n334  \r\n \r\n250

In [31]:
type(finetuning_data[0])

dict

In [25]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf", use_auth_token=True)

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\transformers\models\auto\tokenization_auto.py:690: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


In [26]:
def tokenize_function(examples):
    # Check for 'instruction', 'input', and 'output' keys
    if "instruction" in examples and "input" in examples and "output" in examples:
        text = examples["instruction"][0] + " " + examples["input"][0] + " " + examples["output"][0]
    elif "question" in examples and "answer" in examples:
        text = examples["question"][0] + " " + examples["answer"][0]
    elif "text" in examples:
        text = examples["text"][0]
    else:
        raise ValueError("Input data format not recognized")

    # Assuming 'tokenizer' is a pre-defined tokenizer object
    tokenizer.pad_token = tokenizer.eos_token
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        padding=True,
    )

    max_length = min(
        tokenized_inputs["input_ids"].shape[1],
        2048
    )
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=max_length
    )

    return tokenized_inputs


In [29]:
finetuning_data

[{'instruction': 'What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.',
  'input': 'Table of Contents \r\n3M Company and Subsidiaries\r\nConsolidated Statement of Cash Flow s\r\nYears ended December 31\r\n \r\n(Millions)\r\n    \r\n2018\r\n    \r\n2017\r\n    \r\n2016\r\n \r\nCash Flows from Operating Activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nNet income including noncontrolling interest\r\n \r\n$\r\n5,363  \r\n$\r\n4,869  \r\n$\r\n5,058  \r\nAdjustments to reconcile net income including noncontrolling interest to net cash\r\nprovided by operating activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nDepreciation and amortization\r\n \r\n \r\n1,488  \r\n \r\n1,544  \r\n \r\n1,474  \r\nCompany pension and postretirement contributions\r\n \r\n \r\n(370) \r\n \r\n(967) \r\n \r\n(383) \r\nCompany pension and postretirement expense\r\n \r\n \r\n410  \r\n \r\n334  \r\n \r\n2

In [72]:
from datasets import load_dataset

In [70]:
tokenized_dataset = [tokenize_function(item) for item in finetuning_data]

In [35]:
type(tokenized_dataset)

list

In [73]:
train_dataset = tokenized_dataset.train_test_split(test_size=0.1)['train']
validation_dataset = tokenized_dataset.train_test_split(test_size=0.1)['test']

AttributeError: 'list' object has no attribute 'train_test_split'

In [36]:
tokenized_dataset[0]

{'input_ids': array([[  1, 399, 323, 395]]), 'attention_mask': array([[1, 1, 1, 1]])}

In [43]:
finetuning_data

[{'instruction': 'What is the FY2018 capital expenditure amount (in USD millions) for 3M? Give a response to the question by relying on the details shown in the cash flow statement.',
  'input': 'Table of Contents \r\n3M Company and Subsidiaries\r\nConsolidated Statement of Cash Flow s\r\nYears ended December 31\r\n \r\n(Millions)\r\n    \r\n2018\r\n    \r\n2017\r\n    \r\n2016\r\n \r\nCash Flows from Operating Activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nNet income including noncontrolling interest\r\n \r\n$\r\n5,363  \r\n$\r\n4,869  \r\n$\r\n5,058  \r\nAdjustments to reconcile net income including noncontrolling interest to net cash\r\nprovided by operating activities\r\n \r\n \r\n  \r\n \r\n  \r\n \r\n  \r\nDepreciation and amortization\r\n \r\n \r\n1,488  \r\n \r\n1,544  \r\n \r\n1,474  \r\nCompany pension and postretirement contributions\r\n \r\n \r\n(370) \r\n \r\n(967) \r\n \r\n(383) \r\nCompany pension and postretirement expense\r\n \r\n \r\n410  \r\n \r\n334  \r\n \r\n2

In [53]:
import json
jsonl_file_path = 'finetuning_data.jsonl'

# Writing data to a JSONL file
with open(jsonl_file_path, 'w') as file:
    for item in finetuning_data:
        json.dump(item, file)
        file.write('\n')

In [54]:
model_name="meta-llama/Llama-2-7b-chat-hf"

In [78]:
training_config = {
    "model": {
        "pretrained_name": model_name,
        "max_length" : 2048
    },
    "datasets": {
        "use_hf": False,
        "path": jsonl_file_path
    },
    "verbose": True
}

In [118]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
# Ensure that the tokenizer has a padding token set, using the EOS token if necessary
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# Tokenize the dataset
def tokenize_function(examples):
    # Concatenate 'instruction' and 'input' fields to form the input sequence
    input_sequences = [instr + " " + inp for instr, inp in zip(examples['instruction'], examples['input'])]

    # Tokenize the input sequences
    model_inputs = tokenizer(input_sequences, max_length=512, truncation=True, padding='max_length')

    # Tokenize the output sequences (targets)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['output'], max_length=512, truncation=True, padding='max_length')

    # Assign the tokenized outputs to the 'labels' field
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

# Apply the function to your dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)


# Load dataset from JSONL file
dataset = load_dataset('json', data_files='finetuning_data.jsonl', split='train')

# Apply tokenization to the entire dataset
tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Split the tokenized dataset into training and validation sets
split_datasets = tokenized_datasets.train_test_split(test_size=0.33)
train_dataset = split_datasets['train']
validation_dataset = split_datasets['test']

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

C:\Users\spurt\AppData\Roaming\Python\Python39\site-packages\transformers\tokenization_utils_base.py:3860: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

[codecarbon INFO @ 13:52:35] Energy consumed for RAM : 0.003377 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:52:35] Energy consumed for all CPUs : 0.006023 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:52:35] 0.009400 kWh of electricity used since the beginning.


In [119]:
train_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 100
})

[codecarbon INFO @ 13:52:50] Energy consumed for RAM : 0.003477 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:52:50] Energy consumed for all CPUs : 0.006200 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:52:50] 0.009677 kWh of electricity used since the beginning.


In [86]:
from transformers import AutoModelForCausalLM

In [88]:
base_model = AutoModelForCausalLM.from_pretrained(model_name)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [90]:
logger = logging.getLogger(__name__)
global_config = None

device_count = torch.cuda.device_count()
if device_count > 0:
    logger.debug("Select GPU device")
    device = torch.device("cuda")
else:
    logger.debug("Select CPU device")
    device = torch.device("cpu")

In [91]:
base_model.to(device)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [112]:
def inference(text, model, tokenizer, max_input_tokens=100000, max_output_tokens=1000):
  # Tokenize
  input_ids = tokenizer.encode(
          text,
          return_tensors="pt",
          truncation=True,
          max_length=max_input_tokens
  )

  # Generate
  device = model.device
  generated_tokens_with_prompt = model.generate(
    input_ids=input_ids.to(device),
    max_length=max_output_tokens
  )

  # Decode
  generated_text_with_prompt = tokenizer.batch_decode(generated_tokens_with_prompt, skip_special_tokens=True)

  # Strip the prompt
  generated_text_answer = generated_text_with_prompt[0][len(text):]

  return generated_text_answer

In [113]:
test_text

'Table of Contents\r\n62\r\nADOBE SYSTEMS INCORPORATED\r\nCONSOLIDATED STATEMENTS OF INCOME\r\n(In thousands, except per share data)\r\n \r\nYears Ended\r\n \r\nDecember 2,\r\n2016\r\nNovember 27,\r\n2015\r\nNovember 28,\r\n2014\r\nRevenue:\r\n \r\nSubscription\r\n$\r\n4,584,833\r\n$\r\n3,223,904\r\n$\r\n2,076,584\r\nProduct\r\n800,498\r\n1,125,146\r\n1,627,803\r\nServices and support\r\n469,099\r\n446,461\r\n442,678\r\nTotal revenue\r\n5,854,430\r\n4,795,511\r\n4,147,065\r\n \r\nCost of revenue:\r\nSubscription\r\n461,860\r\n409,194\r\n335,432\r\nProduct\r\n68,917\r\n90,035\r\n97,099\r\nServices and support\r\n289,131\r\n245,088\r\n189,549\r\nTotal cost of revenue\r\n819,908\r\n744,317\r\n622,080\r\n \r\nGross profit\r\n5,034,522\r\n4,051,194\r\n3,524,985\r\n \r\nOperating expenses:\r\nResearch and development\r\n975,987\r\n862,730\r\n844,353\r\nSales and marketing\r\n1,910,197\r\n1,683,242\r\n1,652,308\r\nGeneral and administrative\r\n577,710\r\n531,919\r\n543,332\r\nRestructuring an

In [115]:
test_text = validation_dataset[2]['input']
print("Question input (test):", test_text)
print(f"Correct answer: {validation_dataset[0]['output']}")
print("Model's answer: ")
print(inference(test_text, base_model, tokenizer))

Question input (test): We expect to incur costs of approximately $700 million in connection with separating Upjohn, of which approximately 90% has been incurred since inception and through the second quarter of 2023. These charges include costs and expenses related to separation of legal entities and transaction costs.
Correct answer: 65.4%
Model's answer: 


KeyboardInterrupt: 

In [120]:
max_steps = 10

trained_model_name = f"financebench_{max_steps}_steps"
output_dir = trained_model_name

In [125]:
training_args = TrainingArguments(

  # Learning rate
  learning_rate=1.0e-5,

  # Number of training epochs
  num_train_epochs=1,

  # Max steps to train for (each step is a batch of data)
  # Overrides num_train_epochs, if not -1
  max_steps=max_steps,

  # Batch size for training
  per_device_train_batch_size=1,

  # Directory to save model checkpoints
  output_dir=output_dir,
  report_to="",

  # Other arguments
  overwrite_output_dir=False, # Overwrite the content of the output directory
  disable_tqdm=False, # Disable progress bars
  eval_steps=120, # Number of update steps between two evaluations
  save_steps=120, # After # steps model is saved
  warmup_steps=1, # Number of warmup steps for learning rate scheduler
  per_device_eval_batch_size=1, # Batch size for evaluation
  evaluation_strategy="steps",
  logging_strategy="steps",
  logging_steps=1,
  optim="adafactor",
  gradient_accumulation_steps = 4,
  gradient_checkpointing=False,

  # Parameters for early stopping
  load_best_model_at_end=True,
  save_total_limit=1,
  metric_for_best_model="eval_loss",
  greater_is_better=False
)

[codecarbon INFO @ 13:56:35] Energy consumed for RAM : 0.004966 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:56:35] Energy consumed for all CPUs : 0.008857 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:56:35] 0.013823 kWh of electricity used since the beginning.


In [122]:
model_flops = (
  base_model.floating_point_ops(
    {
       "input_ids": torch.zeros(
           (1, training_config["model"]["max_length"])
      )
    }
  )
  * training_args.gradient_accumulation_steps
)

print(base_model)
print("Memory footprint", base_model.get_memory_footprint() / 1e9, "GB")
print("Flops", model_flops / 1e9, "GFLOPs")

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Lin

In [123]:
from transformers import Trainer

trainer = Trainer(
    model=base_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
)

[codecarbon INFO @ 13:53:03] [setup] RAM Tracking...
[codecarbon INFO @ 13:53:03] [setup] GPU Tracking...
[codecarbon INFO @ 13:53:03] No GPU found.
[codecarbon INFO @ 13:53:03] [setup] CPU Tracking...
[codecarbon WARNING @ 13:53:03] No CPU tracking mode found. Falling back on CPU constant mode.
[codecarbon INFO @ 13:53:05] Energy consumed for RAM : 0.003577 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:53:05] Energy consumed for all CPUs : 0.006380 kWh. Total CPU Power : 42.5 W
[codecarbon WARNING @ 13:53:05] We saw that you have a 12th Gen Intel(R) Core(TM) i7-1255U but we don't know it. Please contact us.
[codecarbon INFO @ 13:53:05] 0.009957 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:53:05] CPU Model on constant consumption mode: 12th Gen Intel(R) Core(TM) i7-1255U
[codecarbon INFO @ 13:53:05] >>> Tracker's metadata:
[codecarbon INFO @ 13:53:05]   Platform system: Windows-10-10.0.22631-SP0
[codecarbon INFO @ 13:53:05]   Python version: 3.9.18
[co

In [134]:
training_output = trainer.train()
output_dir = trained_model_name
save_dir = f'{output_dir}/final'

trainer.save_model(save_dir)
print("Saved model to:", save_dir)

AttributeError: 'AcceleratorState' object has no attribute 'distributed_type'

[codecarbon INFO @ 13:59:50] Energy consumed for RAM : 0.006258 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:59:50] Energy consumed for all CPUs : 0.011160 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:59:50] 0.017418 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:00:05] Energy consumed for RAM : 0.006357 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:00:05] Energy consumed for all CPUs : 0.011337 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:00:05] 0.017695 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:00:20] Energy consumed for RAM : 0.006457 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:00:20] Energy consumed for all CPUs : 0.011514 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:00:20] 0.017971 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:00:35] Energy consumed for RAM : 0.006556 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:00:35] Energy consumed for all C

[codecarbon INFO @ 14:07:05] Energy consumed for all CPUs : 0.016297 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:07:05] 0.025436 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:07:20] Energy consumed for RAM : 0.009238 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:07:20] Energy consumed for all CPUs : 0.016475 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:07:20] 0.025713 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:07:35] Energy consumed for RAM : 0.009337 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:07:35] Energy consumed for all CPUs : 0.016652 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:07:35] 0.025989 kWh of electricity used since the beginning.
[codecarbon INFO @ 14:07:50] Energy consumed for RAM : 0.009437 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 14:07:50] Energy consumed for all CPUs : 0.016829 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 14:07:50] 0.026266 kWh of electricit

In [129]:
!pip install --upgrade transformers accelerate

[codecarbon INFO @ 13:58:20] Energy consumed for RAM : 0.005662 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:58:20] Energy consumed for all CPUs : 0.010097 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:58:20] 0.015759 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:58:35] Energy consumed for RAM : 0.005761 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:58:35] Energy consumed for all CPUs : 0.010274 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:58:35] 0.016035 kWh of electricity used since the beginning.
[codecarbon INFO @ 13:58:50] Energy consumed for RAM : 0.005860 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:58:50] Energy consumed for all CPUs : 0.010451 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:58:50] 0.016312 kWh of electricity used since the beginning.


  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/126.8 kB ? eta -:--:--
     --- ------------------------------------ 10.2/126.8 kB ? eta -:--:--
     -------------------------------------- 126.8/126.8 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.2 MB ? eta -:--:--
   -- ------------------------------------- 0.5/8.2 MB 10.0 MB/s eta 0:00:01
   ----- ---------------------------------- 1.1/8.2 MB 12.1 MB/s eta 0:00:01
   --------- ------------------------------ 2.0/8.2 MB 14.4 MB/s eta 0:00:01
   --------------- ------------------------ 3.1/8.2 MB 16.5 MB/s eta 0:00:01
   --------------------- ------------------ 4.5/8.2 MB 19.0 MB/s eta 0:00:01
   ------------------------------ --------- 6.2/8.2 MB 21.9 MB/s eta 0:00:01
   -----------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autotrain-advanced 0.6.79 requires packaging==23.1, but you have packaging 23.2 which is incompatible.
autotrain-advanced 0.6.79 requires transformers==4.36.1, but you have transformers 4.36.2 which is incompatible.
autotrain-advanced 0.6.79 requires werkzeug==2.3.6, but you have werkzeug 3.0.1 which is incompatible.
[codecarbon INFO @ 13:59:05] Energy consumed for RAM : 0.005960 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:59:05] Energy consumed for all CPUs : 0.010628 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:59:05] 0.016588 kWh of electricity used since the beginning.


In [127]:
!pip check

autotrain-advanced 0.6.79 has requirement packaging==23.1, but you have packaging 23.2.
autotrain-advanced 0.6.79 has requirement werkzeug==2.3.6, but you have werkzeug 3.0.1.


[codecarbon INFO @ 13:58:05] Energy consumed for RAM : 0.005562 kWh. RAM Power : 23.8845477104187 W
[codecarbon INFO @ 13:58:05] Energy consumed for all CPUs : 0.009920 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 13:58:05] 0.015482 kWh of electricity used since the beginning.


In [ ]:
from llama import BasicModelRunner

model = BasicModelRunner("EleutherAI/pythia-410m") 
model.load_data_from_jsonlines("lamini_docs.jsonl", input_key="question", output_key="answer")
model.train(is_public=True)
